In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 1
%pwd

'/ocean/projects/asc170022p/mtragoza/lung-project/notebooks'

In [2]:
import sys, os
import numpy as np
import xarray as xr

sys.path.append('..')
import project

--------------------------------------------------------------------------
but there are no active ports detected (or Open MPI was unable to use
them).  This is most certainly not what you wanted.  Check your
cables, subnet manager configuration, etc.  The openib BTL will be
ignored for this job.

  Local host: dv002
--------------------------------------------------------------------------


RuntimeError: Could not find DOLFIN pkg-config file. Please make sure appropriate paths are set.

In [ ]:
ds = project.Emory4DCTCase(data_root='data/Emory-4DCT')
ds.load_niftis()

In [ ]:
def register_corrfield(case, fixed_phase, moving_phase):

    image_dir = case.nifti_dir.resolve()
    mask_dir = case.mask_dir.resolve()
    output_dir = case.case_dir.resolve() / 'CorrField'
    os.makedirs(output_dir, exist_ok=True)
    
    fixed_name = f'case{case.case_id}_T{fixed_phase:02d}'
    fixed_image = image_dir / (fixed_name + '.nii.gz')
    fixed_mask = mask_dir / fixed_name / 'lung_combined_mask.nii.gz'
    
    moving_name = f'case{case.case_id}_T{moving_phase:02d}'
    moving_image = image_dir / (moving_name + '.nii.gz')
    
    reg_name = f'case{case.case_id}_T{moving_phase:02d}_T{fixed_phase:02d}'
    output_path = output_dir / reg_name

    try:
        cwd = os.getcwd()
        os.chdir('../Lung250M-4B')
        import corrfield
        args = corrfield.parse_args(f'-F {fixed_image} -M {moving_image} -m {fixed_mask} -O {output_path}'.split())
        corrfield.main(args)
    except:
        os.chdir(cwd)
        raise
    finally:
        os.chdir(cwd)

In [ ]:
%pwd

In [ ]:
register_corrfield(case, moving_phase=50, fixed_phase=0)

In [ ]:
import nibabel as nib
import xarray as xr
import hvplot.xarray

a = nib.load('/ocean/projects/asc170022p/mtragoza/lung-project/data/Emory-4DCT/Case1Pack/CorrField/case1_T00_T50.nii.gz')

xr.DataArray(
    a.get_fdata().astype(np.float32)[0],
    dims=['x', 'y', 'z', 'component'],
    coords={
        'x': np.arange(256) * 0.97,
        'y': np.arange(256) * 0.97,
        'z': np.arange(94) * 2.5,
        'component': ['x', 'y', 'z']
    }
).hvplot(groupby=['y', 'component'], x='x', y='z', data_aspect=1, clim=(-10, 10), cmap='seismic')

In [ ]:
case.view(groupby=['phase', 'y'], x='x', y='z')

In [ ]:
%%time

spatial_shape = [case.array.sizes[d] for d in 'xyz']
spatial_coords = [case.array.coords[d] for d in 'xyz']

input_points = utils.nd_coords(spatial_coords)
output_points = F.apply(input_points)
U = output_points - input_points

displacement = xr.DataArray(
    U.reshape(spatial_shape + [3]),
    dims=['x', 'y', 'z', 'component'],
    coords={
        'x': case.array.x,
        'y': case.array.y,
        'z': case.array.z,
        'component': ['x', 'y', 'z']
    }
)
displacement

In [ ]:
data.view_array(displacement, groupby=['y', 'component'], x='x', y='z')

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
F.affine_grid(torch.eye(2, 3).unsqueeze(0), (1, 1, 3, 3), align_corners=True).shape

In [ ]:
img_mov_warped = F.grid_sample(
    img_mov,
    F.affine_grid(
        torch.eye(3, )
    )
)

In [ ]:
sys.path.append('../Lung250M-4B')
import corrfield

corrfield.utils.

In [ ]:


F.grid_sample(
    input=torch.zeros(1,1,),
    grid=torch.zeros(3),
    mode='bilinear'
).shape